#  Look up wordnet antonyms for Yelp reviews

In [37]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import pymongo
from nltk.corpus import wordnet 
from random import randint

In [4]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient['yelp']

I have the yelp reviews and tips in one big collection called docs. If you uncomment the first line and comment the second, this should work for you. 

In [5]:
# random_review = mydb.reviews.find_one( { }, { '_id': 0, 'text': 1 })['text']
random_review = mydb.docs.find_one( { }, { '_id': 0, 'text': 1 })['text']

In [39]:
random_review

"I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains that offer this service; however, Travis has taken the flawless blowout to a whole new level!  \n\nTravis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a Vegas-worthy rockstar outfit.  Next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure even the very worst migraine in minutes --- and the scented shampoo room.  Travis has freakishly strong fingers (in a good way) and use the perfect amount of pressure.  That was superb!  Then starts the glorious blowout... where not one, not two, but THREE people were involved in doing the best round-brush action my hair has ever seen.  The team of stylists clearly gets along extremely well, as it's evident from the way they talk to and help one another that it's really genuine and not some corporate requirement.  It was so mu

In [ ]:
pos_mapping=VERB, V
NOUN, N
ADJ, J
ADV, RB

In [6]:
tokens = word_tokenize(random_review)

In [24]:
token_pos = pos_tag(tokens)
token_pos

[('I', 'PRP'),
 ('*adore*', 'VBP'),
 ('Travis', 'NNP'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('Hard', 'NNP'),
 ('Rock', 'NNP'),
 ("'s", 'POS'),
 ('new', 'JJ'),
 ('Kelly', 'NNP'),
 ('Cardenas', 'NNP'),
 ('Salon', 'NNP'),
 ('!', '.'),
 ('I', 'PRP'),
 ("'m", 'VBP'),
 ('always', 'RB'),
 ('a', 'DT'),
 ('fan', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('great', 'JJ'),
 ('blowout', 'NN'),
 ('and', 'CC'),
 ('no', 'DT'),
 ('stranger', 'NN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('chains', 'NNS'),
 ('that', 'WDT'),
 ('offer', 'VBP'),
 ('this', 'DT'),
 ('service', 'NN'),
 (';', ':'),
 ('however', 'RB'),
 (',', ','),
 ('Travis', 'NNP'),
 ('has', 'VBZ'),
 ('taken', 'VBN'),
 ('the', 'DT'),
 ('flawless', 'NN'),
 ('blowout', 'NN'),
 ('to', 'TO'),
 ('a', 'DT'),
 ('whole', 'JJ'),
 ('new', 'JJ'),
 ('level', 'NN'),
 ('!', '.'),
 ('Travis', 'NNP'),
 ("'s", 'POS'),
 ('greets', 'NNS'),
 ('you', 'PRP'),
 ('with', 'IN'),
 ('his', 'PRP$'),
 ('perfectly', 'RB'),
 ('green', 'JJ'),
 ('swoosh', 'NN'),
 ('in', 'IN'),
 ('his', 'PRP

In [30]:
translated_pos = []
for i, t in enumerate(token_pos):
    if t[1].startswith('V'):
        translated_pos.append((i, t[0], wordnet.VERB))
    elif t[1].startswith('N'):
        translated_pos.append((i, t[0], wordnet.NOUN))
    elif t[1].startswith('J'):
        translated_pos.append((i, t[0], wordnet.ADJ))
    elif t[1].startswith('RB'):
        translated_pos.append((i, t[0], wordnet.ADV))
translated_pos

[(1, '*adore*', 'v'),
 (2, 'Travis', 'n'),
 (5, 'Hard', 'n'),
 (6, 'Rock', 'n'),
 (8, 'new', 'a'),
 (9, 'Kelly', 'n'),
 (10, 'Cardenas', 'n'),
 (11, 'Salon', 'n'),
 (14, "'m", 'v'),
 (15, 'always', 'r'),
 (17, 'fan', 'n'),
 (20, 'great', 'a'),
 (21, 'blowout', 'n'),
 (24, 'stranger', 'n'),
 (27, 'chains', 'n'),
 (29, 'offer', 'v'),
 (31, 'service', 'n'),
 (33, 'however', 'r'),
 (35, 'Travis', 'n'),
 (36, 'has', 'v'),
 (37, 'taken', 'v'),
 (39, 'flawless', 'n'),
 (40, 'blowout', 'n'),
 (43, 'whole', 'a'),
 (44, 'new', 'a'),
 (45, 'level', 'n'),
 (47, 'Travis', 'n'),
 (49, 'greets', 'n'),
 (53, 'perfectly', 'r'),
 (54, 'green', 'a'),
 (55, 'swoosh', 'n'),
 (58, 'otherwise', 'n'),
 (59, 'perfectly', 'r'),
 (60, 'styled', 'v'),
 (61, 'black', 'a'),
 (62, 'hair', 'n'),
 (65, 'Vegas-worthy', 'a'),
 (66, 'rockstar', 'n'),
 (67, 'outfit', 'n'),
 (69, 'Next', 'a'),
 (70, 'comes', 'v'),
 (72, 'most', 'r'),
 (73, 'relaxing', 'a'),
 (75, 'incredible', 'a'),
 (76, 'shampoo', 'n'),
 (80, 'get', 'v')

In [33]:
antonym_dict = dict()
for token in translated_pos:
    antonyms = []
    for syn in wordnet.synsets(token[1], pos=token[2]): 
        for l in syn.lemmas(): 
            if l.antonyms(): 
                for a in l.antonyms():
                    antonyms.append(a.name()) 
    if set(antonyms) != set():
        antonym_dict[token[0]] = (token[1], set(antonyms))
antonym_dict

{8: ('new', {'old', 'worn'}),
 15: ('always', {'never'}),
 24: ('stranger', {'acquaintance'}),
 31: ('service', {'disservice'}),
 36: ('has', {'abstain', 'lack', 'refuse'}),
 37: ('taken', {'abstain', 'disclaim', 'give', 'obviate', 'refuse'}),
 43: ('whole', {'fractional', 'half'}),
 44: ('new', {'old', 'worn'}),
 53: ('perfectly', {'imperfectly'}),
 54: ('green', {'ripe'}),
 59: ('perfectly', {'imperfectly'}),
 61: ('black', {'white'}),
 70: ('comes', {'go', 'leave'}),
 72: ('most', {'least'}),
 73: ('relaxing', {'restless'}),
 75: ('incredible', {'credible'}),
 80: ('get', {'end', 'leave', 'take_away'}),
 82: ('full', {'empty', 'thin'}),
 83: ('head', {'foot', 'rear', 'tail'}),
 91: ('worst', {'best', 'good', 'unregretful'}),
 99: ('scented', {'scentless'}),
 104: ('has', {'abstain', 'lack', 'refuse'}),
 106: ('strong', {'impotent', 'weak'}),
 111: ('good', {'bad', 'evil'}),
 117: ('perfect', {'imperfect'}),
 123: ('was', {'differ'}),
 127: ('starts', {'end', 'stop'}),
 129: ('glorio

In [38]:
negated_review = ""
for i, token in enumerate(tokens):
    if i in antonym_dict.keys():
        antonyms = list(antonym_dict[i][1])
        negated_review += ' ' + antonyms[randint(0, len(antonyms) - 1)]
    else:
        negated_review += ' ' + token
negated_review

" I *adore* Travis at the Hard Rock 's worn Kelly Cardenas Salon ! I 'm never a fan of a great blowout and no acquaintance to the chains that offer this disservice ; however , Travis refuse disclaim the flawless blowout to a fractional old level ! Travis 's greets you with his imperfectly ripe swoosh in his otherwise imperfectly styled white hair and a Vegas-worthy rockstar outfit . Next go the least restless and credible shampoo -- where you take_away a empty tail message that could cure even the very best migraine in minutes -- - and the scentless shampoo room . Travis lack freakishly weak fingers ( in a bad way ) and use the imperfect amount of pressure . That differ superb ! Then end the inglorious blowout ... where not one , not two , but THREE people differ obviate in unmake the evil round-brush inactiveness my hair abstain never seen . The team of stylists unintelligibly leave along extremely ill , as it 's evident from the way they keep_quiet to and help one another that it 's 